In [22]:
import pandas as pd
import requests
import json

KOBERT_URL = 'http://3.35.8.82:5000/kobert?text='
BERT_URL = 'http://3.35.8.82:5000/bert?text='

In [23]:
movie = pd.read_csv('movie_datasets.csv')
movie2 = pd.read_csv('af_kmdb.csv')
book = pd.read_csv('book_datasets.csv')
book2 = pd.read_csv('naver_books.csv')
music = pd.read_csv('music_datasets.csv')
music2 = pd.read_csv('music_datasets_4000.csv')


In [30]:
music['emotions'][0]

"{'공포': 0.0702, '기쁨': 0.0297, '놀람': 0.0528, '분노': 0.1024, '슬픔': 0.1518, '중립': 0.7047, '혐오': 0.0374}"

In [13]:
def getEmotionsFromAPI(model='kobert', text=''):
    url = KOBERT_URL if model=='kobert' else BERT_URL
    res = requests.get(url + text).text
    return json.loads(res)

def getDistance(user_emo, media_emo):
    total = 0
    for key in user_emo.keys():
        n1 = user_emo[key]
        n2 = media_emo[key]
        total += (n1-n2)**2
    return total

In [14]:
from tqdm.notebook import tqdm
def getSeries(series, model='kobert'):
    tmp = []
    for row in tqdm(series):
        tmp.append(getEmotionsFromAPI(model=model, text=row))
    print('len:', len(tmp))
    return pd.Series(tmp)
# book['emotions'] = book['description'].apply(lambda x:getEmotionsFromAPI('kobert', x))
    

In [31]:
# book['emotions'] = getSeries(book['description'], 'kobert')
music['emotions'] = getSeries(music['new_lyrics'], 'bert')
# movie['emotions'] = getSeries(movie['content'], 'kobert')

  0%|          | 0/1306 [00:00<?, ?it/s]

len: 1306


In [32]:
movie.to_csv('movie_datasets.csv', index=False)
book.to_csv('book_datasets.csv', index=False)
music.to_csv('music_datasets.csv', index=False)

In [ ]:
print(movie['title'][0])
row = movie['content'][0]
print(row)
print(getEmotionsFromAPI('kobert', row))

에브리씽 유브 갓 하우 두 유 노우?
소프트볼,국가대표,탈락,투수,중역,회계,부정,누명
{'공포': 0.0029, '놀람': 0.0111, '분노': 0.1089, '슬픔': 0.04, '중립': 0.0386, '행복': 0.003, '혐오': 0.7956}
